In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./archive/movies_metadata.csv')

In [3]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
df = df[['title','genres', 'release_date', 'runtime', 'vote_average','vote_count']]
df.head()

,title,genres,release_date,runtime,vote_average,vote_count
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,81.0,7.7,5415.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,104.0,6.9,2413.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,101.0,6.5,92.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,127.0,6.1,34.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,106.0,5.7,173.0


In [5]:
#Convert release_date into pandas datetime format
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
#Extract year from the datetime
df['year'] = df['release_date'].apply(lambda x: str(x).split('-')[0] if x !=
np.nan else np.nan)

In [6]:
def convert_int(x):
    try:
        return int(x)
    except:
        return 0
df['year'] = df['year'].apply(convert_int)

In [7]:
df.drop(columns='release_date', axis=1, inplace=True)

In [8]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",81.0,7.7,5415.0,1995
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",127.0,6.1,34.0,1995
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",106.0,5.7,173.0,1995


In [9]:
df.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [10]:
df['genres'] = df['genres'].fillna('[]')

In [11]:
import ast
df['genres'] = df['genres'].apply(ast.literal_eval)

In [12]:
df['genres'] = df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [13]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"[Animation, Comedy, Family]",81.0,7.7,5415.0,1995
1,Jumanji,"[Adventure, Fantasy, Family]",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"[Romance, Comedy]",101.0,6.5,92.0,1995
3,Waiting to Exhale,"[Comedy, Drama, Romance]",127.0,6.1,34.0,1995
4,Father of the Bride Part II,[Comedy],106.0,5.7,173.0,1995


In [14]:
s = df.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)

In [15]:
s

0        Animation
0           Comedy
0           Family
1        Adventure
1          Fantasy
           ...    
45461       Family
45462        Drama
45463       Action
45463        Drama
45463     Thriller
Length: 91106, dtype: object

In [16]:
s.name= 'genre'

In [17]:
gen_df = df.drop('genres', axis=1).join(s)
gen_df.head()

,title,runtime,vote_average,vote_count,year,genre
0,Toy Story,81.0,7.7,5415.0,1995,Animation
0,Toy Story,81.0,7.7,5415.0,1995,Comedy
0,Toy Story,81.0,7.7,5415.0,1995,Family
1,Jumanji,104.0,6.9,2413.0,1995,Adventure
1,Jumanji,104.0,6.9,2413.0,1995,Fantasy


In [42]:
def build_cart(gen_df, percentile=0.8):
    genre = 'Animation' # input()
    low_time = 30 # int(input())
    high_time = 120 # int(input())
    
    low_year = 1990 # int(input())
    high_year = 2005 # int(input())
    
    
    movies = gen_df.copy()
    print(movies.shape)
    movies = movies[(movies['genre'] == genre) &
                    (movies['runtime'] >= low_time) &
                    (movies['runtime'] <= high_time) &
                    (movies['year'] >= low_year) &
                    (movies['year'] <= high_year)]
    print(movies.shape)

    C = movies['vote_average'].mean()
    m = movies['vote_count'].quantile(percentile)
    
    q_movies = movies.copy().loc[movies['vote_count'] >= m]
    print(q_movies.shape)
    
    q_movies['score'] = q_movies.apply(lambda x:
            (x['vote_count']/(x['vote_count']+m) * x['vote_average'])
            + (m/(m+x['vote_count']) * C)
            ,axis=1)
    q_movies = q_movies.sort_values('score', ascending=False)
    return q_movies

In [43]:
build_cart(gen_df).head()

(93548, 6)
(439, 6)
(88, 6)


,title,runtime,vote_average,vote_count,year,genre,score
9698,Howl's Moving Castle,119.0,8.2,2049.0,2004,Animation,7.994823
359,The Lion King,89.0,8.0,5520.0,1994,Animation,7.926672
0,Toy Story,81.0,7.7,5415.0,1995,Animation,7.637500
6232,Finding Nemo,100.0,7.6,6292.0,2003,Animation,7.549423
546,The Nightmare Before Christmas,76.0,7.6,2135.0,1993,Animation,7.460500


In [44]:
df.to_csv('./archive/metadata_clean.csv', index=False)